# Hmac python



### Challenge Name: hmac_generate (/embsec/hmac_python/hmac_generate)


    The serial device will send you a variable-length frame of data. The data
    frame will begin with a short, little-endian integer size of the data to 
    follow. You must generate an HMAC-SHA256 signature using the key in 
    'hmackeyfile0.bin'.
    
    The data frame you will receive from the device will be formatted as follows:
    
    [ 0x2   ] [   variable...   ]
    ------------------------------
    | Size   |     Data          |
    ------------------------------
    
    You must send a 32-byte HMAC-SHA256 signature to the serial device.
    
    [     0x20     ]
    ----------------
    |  HMAC(Data)  |
    ----------------
    
    1. Read the HMAC key from 'hmackeyfile0.bin'
    2. Read the size of the data from the serial device
    3. Read the data from the serial device
    4. Generate and send a HMAC-SHA256 over the data
    5. Read the response
    



In [4]:
from embsec import Serial
import struct
from Crypto.Hash import HMAC, SHA256

#https://pycryptodome.readthedocs.io/en/latest/src/hash/hmac.html?highlight=HMAC#Crypto.Hash.HMAC.HMAC.copy 

def hmac_generate():
    ser = Serial("/embsec/hmac_python/hmac_generate")
    # Your code goes here!
    with open("hmackeyfile0.bin", 'rb') as f:  #step 1
        key = f.read()
     
    size = ser.read(2)  #step 2
    size_number = struct.unpack('<h', size)[0]
   # print("size: ", size_number)
    
    data = ser.read(size_number)  #step 3
   # print("data: ", data)
    
    h = HMAC.new(key, data, digestmod=SHA256) #step 4    #generate key
    ser.write(h.digest())    #send over message  #digest finishes making the hmac
    
    print(ser.read_until()) #step 5
    
    
    

hmac_generate()


b'embsec{hmac_generate_19dc2851160268e2}\n'


### Challenge Name: hmac_verify (/embsec/hmac_python/hmac_verify)


    The serial device will send you a series of messages in the format
    described below. For each message you must check the attached signature 
    If verification fails, you must respond with a zero-byte '\x00'. If 
    verification passes, you must respond with a one-byte '\x01'. When the
    length of the message you are about to receive is zero , read a newline-
    terminated flag.

    The serial device will send signed messages in the following format:

    ----------------------------------------
    [ 0x2 ] [  Size bytes    ] [   0x20   ]
    ----------------------------------------
    | Size |     Data         | HMAC(Data) |
    ----------------------------------------



In [2]:
from embsec import Serial
import struct
from Crypto.Hash import HMAC, SHA256


def hmac_verify():
    ser = Serial("/embsec/hmac_python/hmac_verify")
    # Your code goes here!
    
    with open("hmackeyfile0.bin", 'rb') as f: 
            key = f.read()
     
    size = ser.read(2)
    size_number = struct.unpack('<h', size)[0]
   # print("size: ", size_number)
    
    while (size_number != 0):
        data = ser.read(size_number)
        hmac = ser.read(32) #32 because 2 hexadecimal-->32 bits
        
        h = HMAC.new(key, data, digestmod=SHA256)
        
        
        if ((h.digest() ==hmac) == False):  #h.digest() ==hmac is to verify signature
            ser.write(struct.pack('1s', b'\x00'))    #if verification fails send zero-byte

        else:
            ser.write(struct.pack('1s', b'\x01'))  #if verification passes send one-byte
    
        size = ser.read(2)
        size_number = struct.unpack('<h', size)[0]
                      
    print(ser.read_until())
    
    

hmac_verify()


b'embsec{hmac_verify_77572a05cea5fa3a}\n'
